In [1]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt

## Numerics

We want to solve the PDE-constrained shape optimization problem

$$
\min_{\Omega \subset \mathsf{D}} J(u)
:= \int_\Omega |u_\Omega - u_{ref}|^2 \, dx
$$

subject to $(\Omega,u_\Omega)$ satisfying

$$
\int_\Omega \nabla u_\Omega \cdot \nabla v + u_\Omega v \, dx
=
\int_\Omega f v \, dx
\quad \text{for all } v \in H_0^1(\Omega),
$$

where $\Omega \subset \mathbb{R}^2$ and
$u_d, f \in C^1(\mathbb{R}^2)$.


In [2]:
#geo = unit_square.GenerateMesh(maxh=0.05)
geo = OCCGeometry(Circle((0, 0), 1.2).Face(), dim=2).GenerateMesh(maxh=0.1)
mesh = Mesh(geo)
#Draw(mesh)

#f = CoefficientFunction(5 - (x)**2 - (y)**2)
#ud = CF(1 - (x)**2 - (y)**2)


In [3]:
# optimals shape is k lobed flower domain

r = CoefficientFunction((x**2 + y**2)**0.5)
theta = CoefficientFunction(atan2(y, x))

# Anzahl der Blätter
k = 4
# Amplitude der Blätter
A = 0.5

ud = CF(1 - r**2 + A*cos(k*theta))  
f  = CF(5 - r**2) 

grad_f = CoefficientFunction( (f.Diff(x), f.Diff(y) ) )
grad_ud = CoefficientFunction( (ud.Diff(x), ud.Diff(y) ) )

Draw(ud, mesh)

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

### suitable conditions

choosing 

$$
f = 5 - (x-0.5)**2 - (y-0.5)**2 \\

u_d = 1 - (x-0.5)**2 - (y-0.5)**2

$$
should result in a ball as optimals shape

In [4]:
fes = H1(mesh, order=2, dirichlet=".*")
u, v = fes.TnT()
gfu = GridFunction(fes)
scene_gfu = Draw (gfu, mesh, "state")

a = BilinearForm(fes, symmetric=True)
a += grad(u)*grad(v)*dx
a += u*v*dx

fstate = LinearForm(fes)
fstate += f*v*dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [5]:
def SolveStateEquation():
    rhs = gfu.vec.CreateVector()
    rhs.data = fstate.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()) * rhs
    gfu.vec.data += update

In [6]:
a.Assemble()
fstate.Assemble()
SolveStateEquation()
scene_gfu.Redraw()

### Adjoint equation

The adjoint p satisfies

$$
\int_\Omega \nabla w \cdot \nabla p + w p \, dx
=
-2 \int_\Omega (u_\Omega - u_{ref}) w 
\quad \text{for all } w \in H_0^1(\Omega),
$$

In [7]:
def Cost(u):
    return (u - ud)**2 * dx

p, w = fes.TnT()
gfp = GridFunction(fes)
scene_gfp = Draw (gfp, mesh, "adjoint")

fadjoint = LinearForm(fes)
#fadjoint += -1*Cost(gfu).Diff(gfu,w)
fadjoint += -2*(gfu - ud)*w*dx

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [8]:
def SolveAdjointEquation():
    # can use A since diff op is self adjoint
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(fes.FreeDofs()).T * rhs
    gfp.vec.data += update

In [9]:
fadjoint.Assemble()
SolveAdjointEquation()
scene_gfp.Redraw()

The shape derivative of the shape function

$$
\mathcal{J}(\Omega) := \int_\Omega |u - u_d|^2 \, dx
$$

at $\Omega$ in direction $X$ is given by

$$
\begin{aligned}
d\mathcal{J}(\Omega; X)
=& \int_{\Omega} \operatorname{div}(X)\, |u - u_d|^2
- 2 (u - u_d)\, \nabla u_d \cdot X \, dx \\
&+ \int_{\Omega}
\bigl( \operatorname{div}(X) I - D X - D X^\top \bigr)
\nabla u \cdot \nabla p \, dx \\
&+ \int_\Omega \operatorname{div}(X) u p \, dx \\
&- \int_{\Omega}
\bigl( \nabla f \cdot X + f \operatorname{div}(X) \bigr) p \, dx .
\end{aligned}
$$

We now assemble this shape derivative in NGSolve as follows:


In [10]:
#VEC = VectorH1(mesh, order=3, dirichlet=".*")
VEC = VectorH1(mesh, order=2)
PHI, X = VEC.TnT()

dJOmega = LinearForm(VEC)
dJOmega += SymbolicLFI(div(X)*((gfu - ud)**2) - 2*grad_ud*X*(gfu - ud))
dJOmega += SymbolicLFI((div(X)*InnerProduct(grad(gfu),grad(gfp))))
dJOmega += SymbolicLFI((-InnerProduct(X.Deriv()*grad(gfu), grad(gfp))))
dJOmega += SymbolicLFI((-InnerProduct(grad(gfu),X.Deriv()*grad(gfp))))      # Transpose by shifting in scalar product
dJOmega += SymbolicLFI(div(X)*gfu*gfp)
dJOmega += SymbolicLFI(-InnerProduct(grad_f,X)*gfp)
dJOmega += SymbolicLFI(-f*div(X)*gfp)

- Next, we want to find a vector field $X$ which yields a decrease of the objective
  function, i.e. we want to find a vector field $X$ such that

  $$
  d\mathcal{J}(\Omega; X) < 0.
  $$

- This can be achieved by solving an auxiliary boundary value problem of the type

  $$
  \text{Find } X \in H :
  \qquad
  b(X, \Phi) = d\mathcal{J}(\Omega; \Phi)
  \quad \text{for all } \Phi \in H,
  $$

  for a suitable Hilbert space $H$ (e.g. $H = H^1(\Omega)^2$).
  Here, $b(\cdot,\cdot) : H \times H \to \mathbb{R}$ is a positive definite bilinear
  form which we are free to choose. Then $-X$ is a descent direction since

  $$
  d\mathcal{J}(\Omega; -X)
  = - d\mathcal{J}(\Omega; X)
  = - b(X, X)
  < 0.
  $$

- First we start with the bilinear form

  $$
  b(X, Y)
  =
  \int_\Omega \nabla X : \nabla Y + X \cdot Y \, dx .
  $$

- we can also choose X as 
  $$
  \min _{X \in H^1_0(\Omega)} \frac{1}{p} \int _\Omega |\nabla X|^p - DJ(\Omega)(X)
  $$

  since this is a continous functional on $H^1_0(\Omega)'$, it is equivalent to finding $X \in H^1_0(\Omega)$ sucht that, 

  $$
  \int _\Omega |\nabla X | ^{p-2}  \nabla X \cdot \nabla \Phi = DJ(\Omega)(\Phi) \qquad \forall \Phi \in H^1_0(\Omega)
  $$

In [11]:
def pose_deformation_equation(gamma=None): 
    # Bilinearform of Exercise 8/9
    b = BilinearForm(VEC)
    b += InnerProduct(grad(X),grad(PHI))*dx  + InnerProduct(X,PHI)*dx
    if gamma:
        print('Using gamma =', gamma)
        b += gamma * InnerProduct(CF((-X.Diff(x)+X.Diff(y), X.Diff(x)+X.Diff(y))), CF((-PHI.Diff(x)+PHI.Diff(y), PHI.Diff(x)+PHI.Diff(y)))) * dx
    print("Expression tree \n", b)
    return b

def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

    #print("Deformation norm:", sqrt(Integrate(InnerProduct(gfX, gfX), mesh)))

In [12]:
def optimize_shape():
    gfset.Set((0,0))
    mesh.SetDeformation(gfset)
    scene.Redraw()
    a.Assemble()
    fstate.Assemble()
    SolveStateEquation()

    LineSearch = True

    iter_max = 1000
    Jold = Integrate(Cost(gfu), mesh)
    converged = False

    # input("Press enter to start optimization")
    print('Cost at initial design', Integrate (Cost(gfu), mesh))
    for k in range(iter_max):
        mesh.SetDeformation(gfset)
        scene.Redraw()

        print('cost at iteration', k, ': ', Jold)

        a.Assemble()
        fstate.Assemble()
        SolveStateEquation()

        fadjoint.Assemble()
        SolveAdjointEquation()

        b.Assemble()
        #plt.spy(b.mat.ToDense())
        dJOmega.Assemble()
        SolveDeformationEquation()

        scale = 0.01 / Norm(gfX.vec)
        gfsetOld = gfset
        gfset.vec.data -= scale * gfX.vec

        Jnew = Integrate(Cost(gfu), mesh)

        if LineSearch:
            while Jnew > Jold and scale > 1e-12:
                #input('a')
                scale = scale / 2
                print("scale = ", scale)
                #print("scale without normalization factor = ", scale* Norm(gfX.vec))
                if scale <= 1e-12:
                    converged = True
                    break

                gfset.vec.data = gfsetOld.vec - scale * gfX.vec         # scale times -X is descent direction
                mesh.SetDeformation(gfset)                              # applying deformation and solve new state

                a.Assemble()                                           # note that during line search we only have to solve state to check if update yields lower costs              
                fstate.Assemble()
                SolveStateEquation()
                Jnew = Integrate(Cost(gfu), mesh)
                print("New cost during line search: ", Jnew)

        if converged==True:
            print("No more descent can be found")
            break
        Jold = Jnew

        Redraw(blocking=True)

    print("Norm of gset", Integrate(gfset**2, mesh))

In [13]:
b = pose_deformation_equation()
b.Assemble()
gfX = GridFunction(VEC)

# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
scene = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

optimize_shape()

Expression tree 
 on space CompoundFESpaces
symmetric   = 0
multilevel  = 1
nonassemble = 0
printelmat = 0
elmatev    = 0
eliminate_internal = 0
eliminate_hidden = 0
keep_internal = 0
store_inner = 0
integrators: 
  Symbolic BFI
  Symbolic BFI



WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

Cost at initial design 1.2052285904986548
cost at iteration 0 :  1.2052285904986548
cost at iteration 1 :  1.2011454919410245
cost at iteration 2 :  1.1954892270068698
cost at iteration 3 :  1.1898568696779839
cost at iteration 4 :  1.184248392908267
cost at iteration 5 :  1.178663764968244
cost at iteration 6 :  1.1731029501254937
cost at iteration 7 :  1.1675659092259638
cost at iteration 8 :  1.162052600189894
cost at iteration 9 :  1.1565629784347462
cost at iteration 10 :  1.1510969972358456
cost at iteration 11 :  1.1456546080340313
cost at iteration 12 :  1.1402357606983318
cost at iteration 13 :  1.134840403750423
cost at iteration 14 :  1.1294684845566934
cost at iteration 15 :  1.1241199494927758
cost at iteration 16 :  1.118794744084749
cost at iteration 17 :  1.1134928131304538
cost at iteration 18 :  1.1082141008039412
cost at iteration 19 :  1.1029585507455097
cost at iteration 20 :  1.0977261061394616
cost at iteration 21 :  1.092516709781386
cost at iteration 22 :  1.08

In [14]:
# b = pose_deformation_equation(gamma= 1)
# gfX = GridFunction(VEC)

# # gfset denotes the deformation of the original domain and will be updated during the shape optimization
# gfset = GridFunction(VEC)
# gfset.Set((0,0))
# scene = Draw(gfset,mesh,"gfset")
# SetVisualization (deformation=True)

# optimize_shape()

In [15]:
# b = pose_deformation_equation(gamma= 1E2)
# gfX = GridFunction(VEC)

# # gfset denotes the deformation of the original domain and will be updated during the shape optimization
# gfset = GridFunction(VEC)
# gfset.Set((0,0))
# scene = Draw(gfset,mesh,"gfset")
# SetVisualization (deformation=True)

# optimize_shape()